In [40]:
import arcpy
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import math
from arcpy.sa import Raster
from scipy.stats import rankdata
import itertools

In [41]:
gdb_path = r"C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb"
arcpy.env.workspace = gdb_path

# List all feature classes in the geodatabase
feature_classes = arcpy.ListFeatureClasses()
print("Feature Classes in the Geodatabase:")
for fc in feature_classes:
    print(fc)

Feature Classes in the Geodatabase:
WorldImagery_DetectObjectsUs
WorldImagery_DetectObjectsUs1
WorldImagery_DetectObjectsUs2
limits_JSONToFeatures
citylimits_JSONToFeatures
citylimits_JSONToFeatur_Clip
Park_Boundarie_JSONToFeature
Parks_Intersect
Parks_Boundary_out_Intersect
fishnet_park_Intersect
fishnet_park_Inter_Intersect
fishnet_park_Inter_Intersect1
Walkability
fishnet_inter
citylimits
fishnet_Intersect
fishnet_Intersect1
Parks_Boundary_out_Intersect1
Park_Boundarie_JSONToFeature1
Parks_SPR_Merge
Parks_Merge_Intersect
Parks_Merge_Intersect1
fishnet_Intersect2
Parks_Merge_Intersect2
fishnet_sidewalk_intersect
fishnet_sidewalks_Intersect
Seattle_Streets_poly
trails_Intersect
fishnet_ExportFeatures
fishnet_dataframe
Streets_Intersect
Bike_facilities
Bike_facilities_BKF_BL
Bike_facilities_BKF_CLMB
Bike_facilities_BKF_OFFST
Bike_facilities_BKF_SHW
Bike_Greenways_Intersect
Bike_protected_Intersect
Bike_buffer_Intersect
Streets_Intersect1
fishnet_check
fishnet_check_label
fishnet_check_

In [42]:
exported_fishnet_layer = r"C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\walkscore_fishnet"

# Set the workspace to the geodatabase
arcpy.env.workspace = gdb_path

# List all fields in the exported fishnet layer
fields = arcpy.ListFields(exported_fishnet_layer)

# Print field names and types
print("Field Names and Types:")
for field in fields:
    print(f"Name: {field.name}, Type: {field.type}, Alias: {field.aliasName}")

Field Names and Types:
Name: OBJECTID, Type: OID, Alias: OBJECTID
Name: Shape, Type: Geometry, Alias: Shape
Name: FID_walkscore_fishnet, Type: Integer, Alias: FID_walkscore_fishnet
Name: GRID_ID, Type: String, Alias: GRID_ID
Name: IndexID, Type: Integer, Alias: IndexID
Name: total_area, Type: Double, Alias: total_area
Name: Max_Speed_Limit, Type: Double, Alias: MAX_effective_SPEEDLIMIT
Name: SUM_proportional_population, Type: Double, Alias: SUM_proportional_population
Name: Grid_Slope_MEAN, Type: Double, Alias: MEAN
Name: Sidewalks_Slope_Mean, Type: Double, Alias: Sidewalks_Slope_Mean
Name: Streets_Slope_Mean, Type: Double, Alias: Streets_Slope_Mean
Name: MultiUseTrails_Slope_Mean, Type: Double, Alias: MultiUseTrails_Slope_Mean
Name: trails_Slope_Mean, Type: Double, Alias: trails_Slope_Mean
Name: effective_slope, Type: Double, Alias: effective_slope
Name: business_density, Type: Double, Alias: business_density
Name: Industrial_SUM_Industrial_effective_area, Type: Double, Alias: Industr

In [43]:
field_list = []
slope_field = "effective_slope"

In [44]:
for field in arcpy.ListFields(exported_fishnet_layer):
    if "area" in field.name.lower() and field.name.lower() != "shape_area":
        field_list.append(field.name)

# Print the populated field_list
print("fields in walkscore_fishnet:")
for field in field_list:
    print(field)
field_list.append(slope_field)  # Add slope field to the list

fields in walkscore_fishnet:
total_area
Industrial_SUM_Industrial_effective_area
ParkingLots_SUM_ParkingLots_effective_area
GolfCourse_SUM_GolfCourse_effective_area
Cemeteries_SUM_Cemeteries_effective_area
Hospitals_SUM_Hospitals_effective_area
Bike_greenways_SUM_Bike_greenways_area
Bike_protected_SUM_Bike_protected_area
Bike_buffer_SUM_Bike_buffer_area
Healthy_Streets_SUM_Healthy_Streets_area
Parks_SUM_Parks_area
Universities_SUM_Universities_area
Sidewalks_SUM_Sidewalks_area
Plaza_SUM_Plaza_area
trails_SUM_trails_area
MultiUseTrails_SUM_MultiUseTrails_area
Streets_SUM_Streets_effective_area
neighborhood_area
Fragment_Area


In [45]:
def normalize_field(layer, field_name, norm_field_name, inverse=False, fill_null_value=None):
    try:
        # Calculate min and max values for the specified field
        with arcpy.da.SearchCursor(layer, [field_name]) as cursor:
            values = [row[0] for row in cursor if row[0] is not None]
            if not values:
                print(f"No values found for {field_name}. Skipping normalization.")
                return
            min_value = min(values)
            max_value = max(values)
        
        # Add new field for normalized values if it does not already exist
        if not any(f.name == norm_field_name for f in arcpy.ListFields(layer)):
            arcpy.management.AddField(layer, norm_field_name, "DOUBLE")
        
        # Normalize the data and store in the new field
        with arcpy.da.UpdateCursor(layer, [field_name, norm_field_name]) as cursor:
            for row in cursor:
                if row[0] is not None:
                    normalized_value = (row[0] - min_value) / (max_value - min_value)
                    if inverse:
                        normalized_value = 1 - normalized_value
                    row[1] = normalized_value
                else:
                    row[1] = fill_null_value if fill_null_value is not None else None
                cursor.updateRow(row)
        print(f"Normalization complete for {field_name}.")
    except Exception as e:
        print(f"Error processing field {field_name}: {e}")

In [46]:
for field in field_list:
    norm_field = "NORM_" + field
    if field == slope_field:
        normalize_field(exported_fishnet_layer, 'effective_slope', 'NORM_effective_slope', inverse=True, fill_null_value=1)
    else:
        normalize_field(exported_fishnet_layer, field, norm_field)

Normalization complete for total_area.
Normalization complete for Industrial_SUM_Industrial_effective_area.
Normalization complete for ParkingLots_SUM_ParkingLots_effective_area.
Normalization complete for GolfCourse_SUM_GolfCourse_effective_area.
Normalization complete for Cemeteries_SUM_Cemeteries_effective_area.
Normalization complete for Hospitals_SUM_Hospitals_effective_area.
Normalization complete for Bike_greenways_SUM_Bike_greenways_area.
Normalization complete for Bike_protected_SUM_Bike_protected_area.
Normalization complete for Bike_buffer_SUM_Bike_buffer_area.
Normalization complete for Healthy_Streets_SUM_Healthy_Streets_area.
Normalization complete for Parks_SUM_Parks_area.
Normalization complete for Universities_SUM_Universities_area.
Normalization complete for Sidewalks_SUM_Sidewalks_area.
Normalization complete for Plaza_SUM_Plaza_area.
Normalization complete for trails_SUM_trails_area.
Normalization complete for MultiUseTrails_SUM_MultiUseTrails_area.
Normalization co

# Calculating Walkscore

#### Setup

In [47]:
sidewalk_score_field = 'sidewalk_score'
park_score_field = 'park_score'
trail_score_field = 'trail_score'
street_score_field = 'street_score'
bike_score_field = 'bike_score'
walkscore_field = 'walk_score'
unadjusted_walkscore_field = 'unadjusted_walkscore'
slope_field = "effective_slope"

In [48]:
# positive_weights = {
#     sidewalk_score_field: 0.70,
#     park_score_field: 0.175,
#     trail_score_field: 0.1,
#     bike_score_field: 0.025
# }

In [49]:
positive_weights = {
    sidewalk_score_field: 0.5,
    park_score_field: 0.40,
    trail_score_field: 0.05,
    bike_score_field: 0.05
}

In [50]:
negative_weights = {
    "effective_speed_limit_scaler": 1,
}

In [51]:
assert abs(sum(positive_weights.values()) - 1.0) < 1e-6, "The positive weights must sum up to 1.0"

In [52]:
for score_field in [walkscore_field,sidewalk_score_field,park_score_field,trail_score_field,street_score_field,bike_score_field,unadjusted_walkscore_field]:
    if not any(f.name == score_field for f in arcpy.ListFields(exported_fishnet_layer)):
        arcpy.management.AddField(exported_fishnet_layer, score_field, "DOUBLE")

### Slope Scaler

As part of the data, we have a slope element that describes how steep the slope is within each fishnet grid. Instead of handling this element like the others, I'd like to implement a custom function that will allow me to more readily customize the output of the slope scaler.

In [53]:
walkscore_fishnet = exported_fishnet_layer

In [54]:
with arcpy.da.UpdateCursor(walkscore_fishnet, [slope_field]) as cursor:
    for row in cursor:
        if row[0] is None:
            row[0] = 0
        cursor.updateRow(row)

print("Populated null values in Slope field with 0.")

Populated null values in Slope field with 0.


In [55]:
# Field containing the slope values
slope_field = "effective_slope"

# List to store slope values
slope_values = []

# Extract slope values
with arcpy.da.SearchCursor(exported_fishnet_layer, [slope_field]) as cursor:
    for row in cursor:
        if row[0] is not None:
            slope_values.append(row[0])

print(f"Extracted {len(slope_values)} slope values.")

Extracted 10831 slope values.


In [56]:
slope_series = pd.Series(slope_values)
slope_stats = slope_series.describe()
print("Descriptive Statistics for Slope Values:")
print(slope_stats)

Descriptive Statistics for Slope Values:
count    10831.000000
mean         4.095428
std          3.315808
min          0.000000
25%          1.908050
50%          3.365004
75%          5.496817
max         35.683580
dtype: float64


In [57]:
def get_slope_scaler(slope_value):
    if slope_value is None or slope_value < 0:
        return 1.0
    elif slope_value < 2:
        return 1.0
    elif 2 <= slope_value < 3:
        return .9
    elif 3 <= slope_value < 4:
        return 0.7
    elif 4 <= slope_value < 5:
        return 0.5
    elif 5 <= slope_value < 7:
        return 0.3
    elif 7 <= slope_value < 10:
        return 0.1
    elif 10 <= slope_value < 15:
        return 0.01
    elif 15 <= slope_value < 20:
        return 0.00
    elif 20 <= slope_value < 25:
        return 0.0
    else:
        return 0.0

### Streets, ParkingLots, and Industrial Scaler Values

In the previous sheet, I created "effective areas" for each of Streets, Industrial, and Parkinglots. These effective area represent the area of surface streets in each fishnet grid multiplied by the speed limit on each street. Since Industrial and Parklots don't have explicit speed limits, I've set a universal speed value of 15 and used this to calculate the effective area.

Below, I'll create a scaler value based on the sum of all three effective area fields.

In [58]:
walkscore_fishnet_layer = exported_fishnet_layer

In [59]:
# Ensure max_speed_limit_field field exists in walkscore_fishnet_layer
max_speed_limit_field = "Max_Speed_Limit"
if not any(f.name == max_speed_limit_field for f in arcpy.ListFields(walkscore_fishnet_layer)):
    raise ValueError(f"The field {max_speed_limit_field} does not exist in the walkscore_fishnet_layer.")

In [60]:
def get_effective_speed_limit_scaler(max_speed_limit_field):
    if avg_speed_limit <= 0:
        return 1.0
    elif avg_speed_limit < 15:
        return .95
    elif avg_speed_limit < 20:
        return 0.9
    elif avg_speed_limit < 25:
        return 0.85
    elif avg_speed_limit < 30:
        return 0.75
    elif avg_speed_limit < 35:
        return 0.6
    elif avg_speed_limit < 40:
        return 0.4
    elif avg_speed_limit < 45:
        return 0.2
    elif avg_speed_limit < 60:
        return 0.01
    else:
        return 0.00

##### Adding Normalized effective area to fields

In [61]:
streets_field = "Streets_SUM_Streets_effective_area"
industrial_field = "Industrial_SUM_Industrial_effective_area"
parkinglots_field = "ParkingLots_SUM_Parkinglots_effective_area"

In [62]:
combined_values = []

In [63]:
with arcpy.da.SearchCursor(exported_fishnet_layer, [streets_field, industrial_field, parkinglots_field]) as cursor:
    for row in cursor:
        combined_value = (row[0] if row[0] is not None else 0) + \
                         (row[1] if row[1] is not None else 0) + \
                         (row[2] if row[2] is not None else 0)
        combined_values.append(combined_value)

print(f"Extracted {len(combined_values)} combined values.")

Extracted 10831 combined values.


In [64]:
combined_series = pd.Series(combined_values)
combined_stats = combined_series.describe()

In [65]:
combined_series = pd.Series(combined_values)
min_combined = combined_series.min()
max_combined = combined_series.max()

In [66]:
def normalize_combined_value(combined_value):
    return (combined_value - min_combined) / (max_combined - min_combined)

In [67]:
normalized_values = combined_series.apply(normalize_combined_value)

In [68]:
normalized_field = "NORM_Combined_effective_area"

In [69]:
if not any(f.name == normalized_field for f in arcpy.ListFields(exported_fishnet_layer)):
    arcpy.management.AddField(exported_fishnet_layer, normalized_field, "DOUBLE")

In [70]:
with arcpy.da.UpdateCursor(exported_fishnet_layer, [normalized_field]) as cursor:
    for i, row in enumerate(cursor):
        row[0] = normalized_values[i]
        cursor.updateRow(row)

print("Normalized values added to the walkscore_fishnet layer.")

Normalized values added to the walkscore_fishnet layer.


In [71]:
normalized_combined_values = [normalize_combined_value(value) for value in combined_values]
normalized_combined_series = pd.Series(normalized_combined_values)
normalized_combined_stats = normalized_combined_series.describe()

In [72]:
print("Descriptive Statistics for Combined Values:")
print(normalized_combined_stats)

Descriptive Statistics for Combined Values:
count    10831.000000
mean         0.050821
std          0.067004
min          0.000000
25%          0.021823
50%          0.037102
75%          0.055141
max          1.000000
dtype: float64


#### Business Density Scaler

In [73]:
business_density = 'business_density'

In [74]:
# List to store business density values
business_values = []

# Extract business density values
with arcpy.da.SearchCursor(exported_fishnet_layer, [business_density]) as cursor:
    for row in cursor:
        business_values.append(row[0] if row[0] is not None else 0)  # Append 0 for None values

print(f"Extracted {len(business_values)} business density values.")

Extracted 10831 business density values.


In [75]:
business_series = pd.Series(business_values)
business_density_stats = business_series.describe()
print("Descriptive Statistics for Business Values:")
print(business_density_stats)

Descriptive Statistics for Business Values:
count    10831.000000
mean         0.180530
std          0.423916
min          0.000000
25%          0.002866
50%          0.044044
75%          0.163902
max          5.000000
dtype: float64


From the distribution above, we can see that the majority of values are 0. This indicates that most fishnet grids don't have a business in them.

Because of this distribution, I'd like to reward fishnet grids that contain businesses and set the minimum scaler to 1. Below I'll adjust the business density scaler to reflect this, using 1 as the minimum and 2 as the maximum.

In [76]:
def get_business_density_scaler(business_density_value):
    if business_density_value is None or business_density_value <= 0.1:
        return 1.0
    elif 1.5 <= business_density_value <= 5:
        return 2.0 
    elif 1.0 <= business_density_value < 1.5:
        return 1.5
    elif 0.5 <= business_density_value < 1.0:
        return 1.25
    else:
        return 1.0

##### Adding Normalized Business Density to Fields

In [77]:
business_density_field = "business_density"

#### Public Amenity Density Scaler

In [78]:
public_amenity_density = 'amenity_density'

In [79]:
# def get_public_amenity_density_scaler(public_amenity_density_value):
#     if public_amenity_density_value is None or public_amenity_density_value <= 0:
#         return 1.0
#     elif public_amenity_density_value >= 7:
#         return 1.3
#     elif public_amenity_density_value >= 5:
#         return 1.2
#     elif public_amenity_density_value >= 3:
#         return 1.0
#     else:
#         return 1.0

#### Tree Canopy Density Scaler

In [80]:
# def get_canopy_density_scaler(canopy_density_value):
#     if canopy_density_value is None or canopy_density_value <= 0:
#         return 1.0
#     elif canopy_density_value >= 9:
#         return 1.2
#     elif canopy_density_value >= 8:
#         return 1.2
#     elif canopy_density_value >= 7:
#         return 1.1
#     elif canopy_density_value >= 6:
#         return 1.1
#     elif canopy_density_value >= 5:
#         return 1.05
#     elif canopy_density_value >= 4:
#         return 1.0
#     else:
#         return 1.0  # Default case, though it shouldn't normally be reached

#### Crash Density Scaler

In [81]:
crash_values = []
crash_density = 'crash_density_normalized'
# Extract crash values
with arcpy.da.SearchCursor(exported_fishnet_layer, [crash_density]) as cursor:
    for row in cursor:
        if row[0] is not None:
            crash_values.append(row[0] if row[0] is not None else 0)

print(f"Extracted {len(crash_values)} crash Density values.")

Extracted 10831 crash Density values.


In [82]:
crash_series = pd.Series(crash_values)
crash_density_stats = crash_series.describe()
print("Descriptive Statistics for crash Density Values:")
print(crash_density_stats)

Descriptive Statistics for crash Density Values:
count    10831.000000
mean         0.447705
std          0.677248
min          0.000000
25%          0.000108
50%          0.154054
75%          0.625694
max          5.000000
dtype: float64


In [83]:
def get_crash_density_scaler(crash_density_value):
    if crash_density_value is None or crash_density_value < 0.1:
        return 1.0
    elif crash_density_value < .5:
        return 0.95
    elif crash_density_value < 1.5:
        return 0.90
    elif crash_density_value < 3:
        return 0.85
    else:
        return 0.80

#### Crime Density Scaler

In [84]:
def get_crime_density_scaler(crime_density_value):
    if crime_density_value is None or crime_density_value < 0.25:
        return 1.0
    elif crime_density_value < 0.7:
        return 0.95
    elif crime_density_value < 1.5:
        return 0.90
    elif crime_density_value < 3:
        return 0.85
    else:
        return 0.80

### Creating Scaler Fields for Troubleshooting

In [85]:
scaler_fields = {
    'slope_scaler': 'slope_scaler',
    'effective_speed_limit_scaler': 'effective_speed_limit_scaler',
    'business_density_scaler': 'business_density_scaler',
#     'public_amenity_density_scaler': 'public_amenity_density_scaler',
#     'tree_density_scaler':'tree_density_scaler',
    'crash_density_scaler':'crash_density_scaler',
    'crime_density_scaler':'crime_density_scaler'
}

In [86]:
industrial_field = "NORM_Industrial_SUM_Industrial_effective_area"
golfcourses_field = "NORM_GolfCourse_SUM_GolfCourse_effective_area"
hospitals_field = "NORM_Hospitals_SUM_Hospitals_effective_area"
cemeteries_field = "NORM_Cemeteries_SUM_Cemeteries_effective_area"
parkinglots_field = "NORM_ParkingLots_SUM_Parkinglots_effective_area"
streets_field = "NORM_Streets_SUM_Streets_effective_area"
slope_field = "effective_slope"
walkscore_field = "walk_score"
walkscore_positive_field = 'walkscore_positive'
walkscore_negative_field = 'walkscore_negative'
unadjusted_walkscore_field = "unadjusted_walkscore"
# public_amenity_density_field = 'amenity_density'
business_density_field = 'business_density'
# tree_density_field = 'tree_density'
crash_density_field = 'crash_density_normalized'
crime_density_field = 'crime_density_normalized'

In [87]:
score_fields = [sidewalk_score_field, park_score_field, trail_score_field, bike_score_field, walkscore_field, unadjusted_walkscore_field]
# amenities_fields = [business_density, public_amenity_density,tree_density_field,crash_density_field]
amenities_fields = [business_density,crash_density_field,crime_density_field]

In [88]:
fields = [streets_field, industrial_field, golfcourses_field, hospitals_field, cemeteries_field, parkinglots_field, 
          slope_field, walkscore_field, max_speed_limit_field] + list(scaler_fields.values()) + list(score_fields) + list(amenities_fields)

In [89]:
fields

['NORM_Streets_SUM_Streets_effective_area', 'NORM_Industrial_SUM_Industrial_effective_area', 'NORM_GolfCourse_SUM_GolfCourse_effective_area', 'NORM_Hospitals_SUM_Hospitals_effective_area', 'NORM_Cemeteries_SUM_Cemeteries_effective_area', 'NORM_ParkingLots_SUM_Parkinglots_effective_area', 'effective_slope', 'walk_score', 'Max_Speed_Limit', 'slope_scaler', 'effective_speed_limit_scaler', 'business_density_scaler', 'crash_density_scaler', 'crime_density_scaler', 'sidewalk_score', 'park_score', 'trail_score', 'bike_score', 'walk_score', 'unadjusted_walkscore', 'business_density', 'crash_density_normalized', 'crime_density_normalized']

In [90]:
# Add new scaler fields if they don't exist
for field_name in scaler_fields.values():
    if not any(f.name == field_name for f in arcpy.ListFields(walkscore_fishnet)):
        arcpy.management.AddField(walkscore_fishnet, field_name, "DOUBLE")

In [91]:
# Update scalers in walkscore_fishnet
with arcpy.da.UpdateCursor(walkscore_fishnet_layer, fields) as cursor:
    for row in cursor:
        slope_value = row[fields.index(slope_field)]
        business_density_value = row[fields.index(business_density_field)]
        avg_speed_limit = row[fields.index("Max_Speed_Limit")]
        crash_density_value = row[fields.index(crash_density_field)]
        crime_density_value = row[fields.index(crime_density_field)]

        # Handle the case where avg_speed_limit is None
        if avg_speed_limit is None:
            avg_speed_limit = 0  # Set a default value, modify as necessary

        # Calculate individual scalers
        slope_scaler = get_slope_scaler(slope_value)
        effective_speed_limit_scaler = get_effective_speed_limit_scaler(avg_speed_limit)
        business_density_scaler = get_business_density_scaler(business_density_value)
        crash_density_scaler = get_crash_density_scaler(crash_density_value)
        crime_density_scaler = get_crime_density_scaler(crime_density_value)

        # Update row with calculated scalers
        row[fields.index(scaler_fields['slope_scaler'])] = slope_scaler
        row[fields.index(scaler_fields['effective_speed_limit_scaler'])] = effective_speed_limit_scaler
        row[fields.index(scaler_fields['business_density_scaler'])] = business_density_scaler
        row[fields.index(scaler_fields['crash_density_scaler'])] = crash_density_scaler
        row[fields.index(scaler_fields['crime_density_scaler'])] = crime_density_scaler

        cursor.updateRow(row)

print("Scaler values updated and stored in walkscore_fishnet.")

Scaler values updated and stored in walkscore_fishnet.


### Score Component Definitions

In [92]:
sidewalk_score_components = ['NORM_Sidewalks_SUM_Sidewalks_area','NORM_MultiUseTrails_SUM_MultiUseTrails_area',"NORM_Plaza_SUM_Plaza_area"]
park_score_components = ['NORM_Parks_SUM_parks_area', 'NORM_Universities_SUM_Universities_area']
trail_score_components = ['NORM_trails_SUM_trails_area']
bike_score_components = ['NORM_Bike_greenways_SUM_Bike_greenways_area',
                         'NORM_Bike_protected_SUM_Bike_protected_area',
                         'NORM_Bike_buffer_SUM_Bike_buffer_area',
                         'NORM_Healthy_Streets_SUM_Healthy_Streets_area']
street_score_components = ['NORM_Combined_effective_area']
slope_component = [slope_field]

In [93]:
walkscore_fields = (sidewalk_score_components + park_score_components + trail_score_components +
                    bike_score_components + street_score_components + slope_component +
                    score_fields + list(scaler_fields.values()))

In [94]:
for field_name in [walkscore_field, unadjusted_walkscore_field, walkscore_negative_field]:
    if not any(f.name == field_name for f in arcpy.ListFields(walkscore_fishnet)):
        arcpy.management.AddField(walkscore_fishnet, field_name, "DOUBLE")
        walkscore_fields.append(field_name)

In [95]:
# Calculate walkscore using the new effective area scaler
with arcpy.da.UpdateCursor(walkscore_fishnet_layer, walkscore_fields) as cursor:
    for row in cursor:
        # Calculate individual scores
        sidewalk_score = sum(row[walkscore_fields.index(component)] for component in sidewalk_score_components if row[walkscore_fields.index(component)] is not None)
        park_score = sum(row[walkscore_fields.index(component)] for component in park_score_components if row[walkscore_fields.index(component)] is not None)
        trail_score = sum(row[walkscore_fields.index(component)] for component in trail_score_components if row[walkscore_fields.index(component)] is not None)
        bike_score = sum(row[walkscore_fields.index(component)] for component in bike_score_components if row[walkscore_fields.index(component)] is not None)

        row[walkscore_fields.index('sidewalk_score')] = sidewalk_score
        row[walkscore_fields.index('park_score')] = park_score
        row[walkscore_fields.index('trail_score')] = trail_score
        row[walkscore_fields.index('bike_score')] = bike_score

        slope_scaler = row[walkscore_fields.index(scaler_fields['slope_scaler'])]
        effective_speed_limit_scaler = row[walkscore_fields.index(scaler_fields['effective_speed_limit_scaler'])]
        business_density_scaler = row[walkscore_fields.index(scaler_fields['business_density_scaler'])]
        crash_density_scaler = row[walkscore_fields.index(scaler_fields['crash_density_scaler'])]
        crime_density_scaler = row[walkscore_fields.index(scaler_fields['crime_density_scaler'])]

        # Calculate positive component of the walkscore (unadjusted walkscore)
        walkscore_positive = sum(row[walkscore_fields.index(field)] * positive_weights[field] for field in score_fields if row[walkscore_fields.index(field)] is not None)

        # Calculate the negative component of the walkscore
        walkscore_negative = row[walkscore_fields.index('NORM_Combined_effective_area')]
        row[walkscore_fields.index(walkscore_negative_field)] = walkscore_negative

        # Normalize walkscore_positive and walkscore_negative
        max_positive_score = 5  # Change from 10 to 5
        max_negative_score = 5  # Change from 10 to 5

        normalized_positive = walkscore_positive / max_positive_score
        normalized_negative = walkscore_negative / max_negative_score

        # Assign the unadjusted walkscore to the unadjusted_walkscore field
        row[walkscore_fields.index(unadjusted_walkscore_field)] = normalized_positive

        # Calculate final walkscore
#         walkscore = normalized_positive * slope_scaler * effective_speed_limit_scaler * business_density_scaler * public_amenity_density_scaler * tree_density_scaler * crash_density_scaler
        walkscore = normalized_positive * slope_scaler * effective_speed_limit_scaler * business_density_scaler * crash_density_scaler * crime_density_scaler


        # Assign the calculated walkscore to the walkscore field explicitly
        row[walkscore_fields.index(walkscore_field)] = walkscore

        cursor.updateRow(row)

print("Aggregated and scaled walkscore calculation complete.")

Aggregated and scaled walkscore calculation complete.


In [96]:
walkscore_fishnet = r"C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\walkscore_fishnet"
walkscore_field = "walk_score"

# Check if the field exists in the layer
field_names = [field.name for field in arcpy.ListFields(walkscore_fishnet)]
if walkscore_field not in field_names:
    raise ValueError(f"Field '{walkscore_field}' does not exist in the layer.")

# Extract walkscore values
walkscore_values = []

with arcpy.da.SearchCursor(walkscore_fishnet, [walkscore_field]) as cursor:
    for row in cursor:
        walkscore_values.append(row[0] if row[0] is not None else 0)

# Rank transformation
ranks = rankdata(walkscore_values, method='dense')

# Min-max scaling to [0, 100] after ranking
min_rank = min(ranks)
max_rank = max(ranks)

def min_max_scale(value, min_value, max_value, new_min, new_max):
    return new_min + (value - min_value) * (new_max - new_min) / (max_value - min_value)

scaled_scores = [min_max_scale(rank, min_rank, max_rank, 0, 100) for rank in ranks]

# Add scaled scores back to the shapefile
with arcpy.da.UpdateCursor(walkscore_fishnet, [walkscore_field]) as cursor:
    for i, row in enumerate(cursor):
        row[0] = scaled_scores[i]
        cursor.updateRow(row)

print("Walkscore normalization using rank transformation and scaling to [0, 100] complete.")

Walkscore normalization using rank transformation and scaling to [0, 100] complete.


### Aggregating Walkscore by Neighborhood

In [97]:
# Set environment settings
arcpy.env.workspace = r"C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb"
arcpy.env.overwriteOutput = True  # Allow outputs to be overwritten

# Define the input layers
walkscore_fishnet = r"C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\walkscore_fishnet"
neighborhoods = r"C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\neighborhoods"
clipped_walkscore_fishnet = "walkscore_fishnet_clipped"
spatial_join_output = "walkscore_neighborhoods_join"
statistics_output = "walkscore_statistics"
output_layer = "walkscore_neighborhoods"

# Step 1: Ensure the spatial reference systems match
walkscore_sr = arcpy.Describe(walkscore_fishnet).spatialReference
neighborhoods_sr = arcpy.Describe(neighborhoods).spatialReference

if walkscore_sr.name != neighborhoods_sr.name:
    raise ValueError("Spatial references do not match between walkscore_fishnet and neighborhoods.")

# Step 2: Clip the walkscore fishnet to the neighborhoods layer
arcpy.analysis.Clip(walkscore_fishnet, neighborhoods, clipped_walkscore_fishnet)
print("Clipped walkscore fishnet to neighborhoods layer.")

# Step 3: Perform a spatial join to associate each fishnet cell with a neighborhood
arcpy.analysis.SpatialJoin(target_features=clipped_walkscore_fishnet,
                           join_features=neighborhoods,
                           out_feature_class=spatial_join_output,
                           join_type="KEEP_COMMON",
                           match_option="INTERSECT")
print("Performed spatial join between neighborhoods and clipped walkscore fishnet.")

# Step 4: Calculate the total walkscore, total area, and weighted walkscores for each neighborhood
# Add fields for area calculation in the fishnet
arcpy.management.AddGeometryAttributes(spatial_join_output, "AREA_GEODESIC", Area_Unit="SQUARE_FEET_US")
print("Added geometry attributes to calculate area for each fishnet grid.")

# Step 5: Create a new table to calculate weighted sums using the correct area field name
arcpy.analysis.Statistics(spatial_join_output, statistics_output,
                          [["walk_score", "SUM"], ["AREA_GEO", "SUM"], ["walk_score", "FIRST"]],
                          "nested")
print("Calculated statistics for walkscore and area for each neighborhood.")

# Step 6: Create the walkscore_neighborhoods layer by exporting the neighborhoods layer
arcpy.conversion.FeatureClassToFeatureClass(in_features=neighborhoods,
                                            out_path=arcpy.env.workspace,
                                            out_name=output_layer)
print("Created the walkscore_neighborhoods layer.")

# Step 7: Add a field for the weighted average walkscore in the walkscore_neighborhoods layer
weighted_average_walkscore_field = "weighted_avg_walk_score"

# Delete the existing field if it exists
if any(f.name == weighted_average_walkscore_field for f in arcpy.ListFields(output_layer)):
    arcpy.management.DeleteField(output_layer, weighted_average_walkscore_field)

arcpy.management.AddField(output_layer, weighted_average_walkscore_field, "DOUBLE")

# Step 8: Join the statistics table to the walkscore_neighborhoods layer
arcpy.management.JoinField(in_data=output_layer,
                           in_field="nested",
                           join_table=statistics_output,
                           join_field="nested",
                           fields=["SUM_walk_score", "SUM_AREA_GEO"])
print("Joined the statistics table to the walkscore_neighborhoods layer.")

# Step 9: Calculate the weighted average walkscore for each neighborhood with power-normalized area
exponent = 0.85  # Define the exponent for power normalization (less than 1 to reduce the effect of smaller areas)

with arcpy.da.UpdateCursor(output_layer, ["SUM_walk_score", "SUM_AREA_GEO", weighted_average_walkscore_field]) as cursor:
    for row in cursor:
        if row[0] is not None and row[1] is not None and row[1] > 0:
            # Power normalize the area
            normalized_area = math.pow(row[1], exponent)

            # Calculate the weighted average walkscore using the power-normalized area
            row[2] = row[0] / normalized_area  # SUM_walk_score / (SUM_AREA_GEO ^ exponent)
        else:
            row[2] = None
        cursor.updateRow(row)

print("Calculated weighted average walkscore for each neighborhood with power-normalized area.")

# Step 10: Rank-normalize the weighted average walkscores
rank_normalized_walk_score_field = "rank_normalized_walk_score"

# Delete the existing field if it exists
if any(f.name == rank_normalized_walk_score_field for f in arcpy.ListFields(output_layer)):
    arcpy.management.DeleteField(output_layer, rank_normalized_walk_score_field)

arcpy.management.AddField(output_layer, rank_normalized_walk_score_field, "DOUBLE")

# Step 10.1: Extract the weighted average walkscores and rank them
scores = [(row[0], i) for i, row in enumerate(arcpy.da.SearchCursor(output_layer, [weighted_average_walkscore_field])) if row[0] is not None]

# Sort scores and create ranks
sorted_scores = sorted(scores, key=lambda x: x[0])
ranks = [(score[1], i + 1) for i, score in enumerate(sorted_scores)]

# Normalize the ranks to be between 0 and 100
min_rank = min(rank[1] for rank in ranks)
max_rank = max(rank[1] for rank in ranks)
normalized_ranks = [(rank[0], 100 * (rank[1] - min_rank) / (max_rank - min_rank)) for rank in ranks]

# Step 10.2: Assign the rank-normalized scores back to the walkscore_neighborhoods layer
normalized_rank_dict = dict(normalized_ranks)

with arcpy.da.UpdateCursor(output_layer, [weighted_average_walkscore_field, rank_normalized_walk_score_field]) as cursor:
    for i, row in enumerate(cursor):
        if i in normalized_rank_dict:
            row[1] = normalized_rank_dict[i]
        else:
            row[1] = None
        cursor.updateRow(row)
print("Rank-normalized weighted average walkscores to be between 0 and 5.")

# Step 11: Clean up temporary fields
fields_to_delete = ["SUM_walk_score", "SUM_AREA_GEO"]
for field in fields_to_delete:
    if any(f.name == field for f in arcpy.ListFields(output_layer)):
        arcpy.management.DeleteField(output_layer, field)
        print(f"Deleted temporary field: {field}")

print("Temporary fields cleaned up.")

Clipped walkscore fishnet to neighborhoods layer.
Performed spatial join between neighborhoods and clipped walkscore fishnet.
Added geometry attributes to calculate area for each fishnet grid.
Calculated statistics for walkscore and area for each neighborhood.
Created the walkscore_neighborhoods layer.
Joined the statistics table to the walkscore_neighborhoods layer.
Calculated weighted average walkscore for each neighborhood with power-normalized area.
Rank-normalized weighted average walkscores to be between 0 and 5.
Deleted temporary field: SUM_walk_score
Deleted temporary field: SUM_AREA_GEO
Temporary fields cleaned up.


#### Feature Reduction

The end goal of this map is to be a user interface that allows users to interact with the data. To improve performance for the end-user, I'll trim down the fields in walkscore_fishnet and walkscore_neighborhoods to reduce the amount of data that needs to be served to each person that interacts with the UI.

In [98]:
check_fields = arcpy.ListFields(exported_fishnet_layer)

print("Field Names and Types:")
for field in check_fields:
    print(f"Name: {field.name}, Type: {field.type}, Alias: {field.aliasName}")

Field Names and Types:
Name: OBJECTID, Type: OID, Alias: OBJECTID
Name: Shape, Type: Geometry, Alias: Shape
Name: FID_walkscore_fishnet, Type: Integer, Alias: FID_walkscore_fishnet
Name: GRID_ID, Type: String, Alias: GRID_ID
Name: IndexID, Type: Integer, Alias: IndexID
Name: total_area, Type: Double, Alias: total_area
Name: Max_Speed_Limit, Type: Double, Alias: MAX_effective_SPEEDLIMIT
Name: SUM_proportional_population, Type: Double, Alias: SUM_proportional_population
Name: Grid_Slope_MEAN, Type: Double, Alias: MEAN
Name: Sidewalks_Slope_Mean, Type: Double, Alias: Sidewalks_Slope_Mean
Name: Streets_Slope_Mean, Type: Double, Alias: Streets_Slope_Mean
Name: MultiUseTrails_Slope_Mean, Type: Double, Alias: MultiUseTrails_Slope_Mean
Name: trails_Slope_Mean, Type: Double, Alias: trails_Slope_Mean
Name: effective_slope, Type: Double, Alias: effective_slope
Name: business_density, Type: Double, Alias: business_density
Name: Industrial_SUM_Industrial_effective_area, Type: Double, Alias: Industr

In [99]:
fields_to_keep = [
    "OBJECTID", "Shape","NORM_total_area", "IndexID", "Shape_Length", "Shape_Area", "walk_score","effective_slope",
    "unadjusted_walkscore", "slope_scaler", "effective_speed_limit_scaler","business_density_scaler","crash_density_scaler",
    "crime_density_scaler","Max_Speed_Limit","business_density",'crash_density_normalized','crime_density_normalized','nested','latitude','longitude'
]
walkscore_points_layer = r"C:\Users\rtvpd\Documents\Walkability_Seattle\Walkability_Seattle.gdb\walkscore_fishnet"
# List all fields in the walkscore_fishnet_layer
all_fields = arcpy.ListFields(walkscore_points_layer)

# Create a list of fields to delete
fields_to_delete = [field.name for field in all_fields if field.name not in fields_to_keep]

# Delete the unwanted fields
if fields_to_delete:
    arcpy.management.DeleteField(walkscore_points_layer, fields_to_delete)
    print(f"Deleted fields: {fields_to_delete}")
else:
    print("No fields to delete.")

print("Field pruning complete.")

Deleted fields: ['FID_walkscore_fishnet', 'GRID_ID', 'total_area', 'SUM_proportional_population', 'Grid_Slope_MEAN', 'Sidewalks_Slope_Mean', 'Streets_Slope_Mean', 'MultiUseTrails_Slope_Mean', 'trails_Slope_Mean', 'Industrial_SUM_Industrial_effective_area', 'ParkingLots_SUM_ParkingLots_effective_area', 'GolfCourse_SUM_GolfCourse_effective_area', 'Cemeteries_SUM_Cemeteries_effective_area', 'Hospitals_SUM_Hospitals_effective_area', 'Slope_MEAN', 'Bike_greenways_SUM_Bike_greenways_area', 'Bike_protected_SUM_Bike_protected_area', 'Bike_buffer_SUM_Bike_buffer_area', 'Healthy_Streets_SUM_Healthy_Streets_area', 'Parks_SUM_Parks_area', 'Universities_SUM_Universities_area', 'Sidewalks_SUM_Sidewalks_area', 'Plaza_SUM_Plaza_area', 'trails_SUM_trails_area', 'MultiUseTrails_SUM_MultiUseTrails_area', 'Streets_SUM_Streets_effective_area', 'population_SUM_proportional_population', 'SPD_Crime_Data_COUNT_Offense_ID', 'FID_neighborhoods', 'city', 'neighborhood_area', 'is_tourist', 'is_industrial', 'Fragme

In [100]:
final_fields = arcpy.ListFields(walkscore_points_layer)

print("Field Names and Types:")
for field in final_fields:
    print(f"Name: {field.name}, Type: {field.type}, Alias: {field.aliasName}")

Field Names and Types:
Name: OBJECTID, Type: OID, Alias: OBJECTID
Name: Shape, Type: Geometry, Alias: Shape
Name: IndexID, Type: Integer, Alias: IndexID
Name: Max_Speed_Limit, Type: Double, Alias: MAX_effective_SPEEDLIMIT
Name: effective_slope, Type: Double, Alias: effective_slope
Name: business_density, Type: Double, Alias: business_density
Name: nested, Type: String, Alias: nested
Name: latitude, Type: Double, Alias: latitude
Name: longitude, Type: Double, Alias: longitude
Name: Shape_Length, Type: Double, Alias: Shape_Length
Name: Shape_Area, Type: Double, Alias: Shape_Area
Name: crime_density_normalized, Type: Double, Alias: crime_density_normalized
Name: crash_density_normalized, Type: Double, Alias: crash_density_normalized
Name: NORM_total_area, Type: Double, Alias: NORM_total_area
Name: walk_score, Type: Double, Alias: walk_score
Name: unadjusted_walkscore, Type: Double, Alias: unadjusted_walkscore
Name: slope_scaler, Type: Double, Alias: slope_scaler
Name: effective_speed_limi

In [101]:
neighborhood_fields_to_keep = [
    "OBJECTID", "Shape", "city", "nested", "neighborhood_area", "rank_normalized_walk_score","Shape_Length", "Shape_Area",'latitude','longitude'
]
neighborhood_walkscore_layer = "walkscore_neighborhoods"

# List all fields in the walkscore_fishnet_layer
all_fields = arcpy.ListFields(neighborhood_walkscore_layer)

# Create a list of fields to delete
fields_to_delete = [
    field.name for field in all_fields 
    if field.name not in neighborhood_fields_to_keep and not field.name.startswith("SUM_COUNT")
]
# Delete the unwanted fields
if fields_to_delete:
    arcpy.management.DeleteField(neighborhood_walkscore_layer, fields_to_delete)
    print(f"Deleted fields: {fields_to_delete}")
else:
    print("No fields to delete.")

print("Field pruning complete.")

Deleted fields: ['is_tourist', 'is_industrial', 'weighted_avg_walk_score']
Field pruning complete.
